<a href="https://colab.research.google.com/github/jismartin/sheva/blob/main/sheva_monte_carlo_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SHEVA Shapley values for Earned VAlue management

# Monte-Carlo simulation
Monte Carlo simulation of a case study based on Lambrechts (2008)

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy.stats
import plotly.express as px

In [2]:
def create_aon_network(edges_file,activities_file):
    # The activity‐on‐node (AON) network is modeled as a Networkx directed acyclic 
    # edges_file contains the edge list of the network
    # activities_file contains the duration and cost values
    dfnet=pd.read_csv(edges_file)
    dfact=pd.read_csv(activities_file)
    G=nx.DiGraph()
    for i in dfnet.index:
        G.add_edge(dfnet.loc[i,'n1'],dfnet.loc[i,'n2'])
    for i in dfact.index:
        G.nodes[dfact.loc[i,'node']]['t_duration']=0
        G.nodes[dfact.loc[i,'node']]['t_start']=0
        G.nodes[dfact.loc[i,'node']]['duration_mu']=dfact.loc[i,'mu'] # Mean of the Normal distribution
        G.nodes[dfact.loc[i,'node']]['duration_var']=dfact.loc[i,'var'] # Variance of the Normal distribution
        G.nodes[dfact.loc[i,'node']]['cost']=dfact.loc[i,'cost'] # Variable cost
    print(nx.info(G))
    print('Is G aperiodic?',nx.is_aperiodic(G))  
    return G

In [3]:
G=create_aon_network('./data/aon_edges.txt.','./data/aon_activities.txt')

DiGraph with 10 nodes and 12 edges
Is G aperiodic? True


## Functions for simulation

In [4]:
def initialize_graph(G):
    # Reset start time to zero
    for i in G.nodes():
        G.nodes[i]['t_start']=0


In [5]:
def draw_random_durations(G,rand=True,noise='none',n_std=1):
    # for each tick, draw the activities' duration (depending on the experiment)
    if rand==True:
        for i in G.nodes():
            G.nodes[i]['t_duration']=np.random.normal(G.nodes[i]['duration_mu'],
            np.sqrt(G.nodes[i]['duration_var']))
        if noise=='2-5': #
            # Node 2 to Node 5
            if G.nodes[2]['t_duration'] >= (G.nodes[2]['duration_mu']+ (np.sqrt(G.nodes[2]['duration_var']) *n_std)): # n_std deviation
                G.nodes[5]['t_duration']=np.random.normal(G.nodes[5]['duration_mu'] + 12, np.sqrt(G.nodes[5]['duration_var']))
        elif noise=='5-rand':
            th=1-scipy.stats.norm.cdf(G.nodes[2]['duration_mu'] + (np.sqrt(G.nodes[2]['duration_var']) *n_std),
                loc=G.nodes[2]['duration_mu'],
                scale=np.sqrt(G.nodes[2]['duration_var']))
            if np.random.rand()<th:
                G.nodes[5]['t_duration']=np.random.normal(G.nodes[5]['duration_mu'] + 12, np.sqrt(G.nodes[5]['duration_var']))
    else:
        # for checking with the average durations
        for i in G.nodes():
            G.nodes[i]['t_duration']=G.nodes[i]['duration_mu']


In [6]:
def compute_times(G):
    # Solve the start time of activities
    # follow a list of nodes in topologically sorted order
    for i in nx.topological_sort(G):       
        t2=G.nodes[i]['t_start'] + G.nodes[i]['t_duration'] # end time
        for j in G.neighbors(i):
            G.nodes[j]['t_start']=np.max([G.nodes[j]['t_start'],t2])       

In [7]:
def compute_duration_at_EV(G,df):
    # Compute the project duration at a particular EV state
    G2=G.copy()
    initialize_graph(G2)
    for i in [n for n in G2.nodes() if n not in [0,9]]:
        G2.nodes[i]['t_duration']=df.loc[df['node']==i,'duration@'].values[0]
    compute_times(G2)
    return np.max([G2.nodes[i]['t_start'] for i in G2.nodes()])        

In [8]:
def create_df_run(G):
    # dataframe of a run used to compute project's values
    df_run=pd.DataFrame([[ i,
        G.nodes[i]['cost'],
        G.nodes[i]['duration_mu'],
        G.nodes[i]['t_duration'],
        G.nodes[i]['t_start'],
        G.nodes[i]['t_start'] + G.nodes[i]['t_duration'], # end time
        0,0] for i in G.nodes()],
        columns=['node','cost','duration_mu','duration','start','end','duration@','ev@'])
    df_run.index=df_run['node'].values
    df_run.sort_values(by='start',inplace=True)
    return df_run

In [9]:
def evaluate_at_EV(G,EV,Ti=0,N=1000):
    # compute the project status at a particular eraned value (iterative approach)
    df_ev=create_df_run(G)
    A=df_ev.values 
    c={'node':0, 'cost':1, 'duration_mu':2, 'duration':3, 'start':4, 'end':5, 'duration@':6, 'ev@':7}

    # Iterative approach (increase N to get more decimal precission)
    for T in np.linspace(Ti,df_ev['end'].max(),N):
        # Not started activities
        not_started = T<=A[:,c['start']]
        A[not_started,c['duration@']]=0
        A[not_started,c['ev@']]=0

        # Ended activities
        ended = T>=A[:,c['end']]
        A[ended,c['duration@']]=A[ended,c['duration']] # Real duration
        A[ended,c['ev@']]=A[ended,c['duration_mu']]*A[ended,c['cost']] # plan cost

        # Running activities
        running = (A[:,c['start']]<T) & (A[:,c['end']]>T)
        A[running,c['duration@']]= A[running,c['duration']] * (T-A[running,c['start']]) / (A[running,c['end']]-A[running,c['start']]) # % of the real duration
        A[running,c['ev@']]= A[running,c['duration_mu']] * A[running,c['cost']] * (T-A[running,c['start']]) / (A[running,c['end']]-A[running,c['start']]) # % of the plan cost
        if A[:,c['ev@']].sum()/(A[:,c['duration_mu']]*A[:,c['cost']]).sum()>=EV:
            break;
    df_ev.loc[:,'duration@']=A[:,c['duration@']]
    df_ev.loc[:,'ev@']=A[:,c['ev@']]
    return df_ev

In [10]:
def get_mean_time(G):
    # compute the expected duration
    G2=G.copy()
    initialize_graph(G2)
    draw_random_durations(G2,rand=False)
    compute_times(G2)
    return np.max([G2.nodes[i]['t_start'] for i in G2.nodes()])
        

In [ ]:
def get_mean_time(G):
    # compute the expected duration
    G2=G.copy()
    initialize_graph(G2)
    draw_random_durations(G2,rand=False)
    compute_times(G2)
    return np.max([G2.nodes[i]['t_start'] for i in G2.nodes()])

## Experiments (EV = 75% BAC)

In [ ]:
def critical_path(df):
    # Crtitical pathc of the case study project
    paths=[('147',np.sum([df.loc[i,'duration']  for i in [1,4,7]])),
        ('259',np.sum([df.loc[i,'duration']  for i in [2,5,9]])),
        ('268',np.sum([df.loc[i,'duration']  for i in [2,6,8]])),
        ('368',np.sum([df.loc[i,'duration']  for i in [3,6,8]]))]
    return sorted(paths,key=lambda x: -x[1])[0][0]

In [ ]:
def simulation(edges_file,activities_file,experiment,EV,Nruns=50000):
    # Monte-Carlo simulation
    # AON network
    G=create_aon_network(edges_file,activities_file)
    
    sim_list=list() # list of dictionaries with runs
    print('Starting simulation...')
    for m in range(Nruns):
        # Initilize
        initialize_graph(G)
        # Draw durations
        draw_random_durations(G,noise=experiment)
        # Compute times
        compute_times(G)
        # Compute @EV
        df=evaluate_at_EV(G,EV)
        # Save run
        sim={'duration'+str(i):df.loc[i,'duration'] for i in range(1,9)}
        sim.update({'duration@'+str(i):df.loc[i,'duration@'] for i in range(1,9)})
        sim.update({'cost@':(df['cost']*df['duration@']).sum(),
            'duration@':compute_duration_at_EV(G,df),
            'cost':(df['cost']*df['duration']).sum(),
            'duration':df['end'].max(),
            'critical_path':critical_path(df) })
        sim_list.append(sim)
        print(m)
    print('... end simulation')
    pd.DataFrame.from_dict(sim_list).to_csv('./data/simulation_EV'+str(EV)+'_'+experiment+'.csv')

**Null model (5-rand) of comparison**

In [ ]:
# Null model (5-rand) of comparison
simulation(edges_file='./data/aon_edges.txt.',activities_file='./data/aon_activities.txt',experiment='5-rand',EV=0.75)

**Interaction between activities 2-5**

In [ ]:
#  interaction between activities 2-5
simulation(edges_file='./data/aon_edges.txt.',activities_file='./data/aon_activities.txt',experiment='2-5',EV=0.75)